In [ ]:
!pip install -U py-boost

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import os
import sys

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from pandas import Series, DataFrame

import cupy as cp
import cudf
from py_boost import GradientBoosting
from py_boost.gpu.losses import Metric

from sklearn.model_selection import StratifiedKFold

### Preprocessing 

The following script in the hidden cell will read the data with **RAPIDS CuDF** and save to **.feather**. Also it splits the test data into the 4 partitions to prevent OOM during inference. It takes quite a long time (about 30 min) in Kernel probably due to RAM limitations, but locally it takes about 2 mins, so I just use precomputed data. But you may set PREPROCESSING=True if do not want to use the external dataset.

In [ ]:
script = """
import os
import numpy as np
import math
import pandas as pd
import cudf
import subprocess

        
def process_fn(path, **kwargs):
    
    data = cudf.read_csv(path, dtype=DTYPES, **kwargs)
    data['customer_ID'] = data['customer_ID'].map(cudf.from_pandas(MAPPER))
    data['S_2'] = cudf.to_datetime(data['S_2'])

    for col in CATS:
        if col in FILL_DICT:
            data[col] += FILL_DICT[col]
            data[col] = data[col].fillna(0).astype(np.uint8)
        else:
            data[col] = data[col].map(cudf.Series(MAP_DICT[col], dtype=np.uint8)).fillna(0)
            
    return data


def batch_process_fn(path, batch_size, output, start=0, stop=None):
    
    names = pd.read_csv(path, nrows=2).columns.to_list()
    
    # iterate over file
    parts = []
    skiprows = 1 + start
    if stop is None:
        # nrows + 1 
        stop = int(subprocess.check_output(
            ['wc', '-l', path]).decode("utf-8").split(' ')[0]) - 1
        
    total = stop - start
    niter = math.ceil(total / batch_size)
    print(start, stop, total)

    for i in range(niter):
        batch_size_ = min(batch_size, total - i * batch_size)
        
        data = process_fn(path, skiprows=skiprows, nrows=batch_size_, names=names)
        skiprows += batch_size
        data = data.to_pandas()
        parts.append(data)
        
        print(data.shape)

    data = pd.concat(parts, axis=0, ignore_index=True)
    data.to_feather(output)
    print(data.shape)
    

if __name__ == '__main__':
    
    os.makedirs('./data', exist_ok=True)
    
    train_ids = pd.read_csv('../input/amex-default-prediction/train_labels.csv', )
    test_ids = pd.read_csv('../input/amex-default-prediction/sample_submission.csv', )

    ids = pd.concat([train_ids['customer_ID'], test_ids['customer_ID']], axis=0, 
                   ).sort_values().reset_index()
    ids['index'] = ids['index'].astype(np.int32)

    MAPPER = ids.set_index('customer_ID')['index']

    train_ids['my_id'] = train_ids['customer_ID'].map(MAPPER).values
    test_ids['my_id'] = test_ids['customer_ID'].map(MAPPER).values

    train_ids.to_feather('./data/target.feather')
    test_ids.to_feather('./data/ssub.feather')

    # analyze sample
    sample = pd.read_csv('../input/amex-default-prediction/train_data.csv', nrows=100)
    CATS = ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']
    NUMS = sample.columns.drop(['customer_ID', 'S_2'] + CATS).to_list()
    FEATS = sample.columns.to_list()

    FILL_DICT = {
        'B_30': 1, 'B_38': 0, 'D_114': 1, 'D_116': 1, 'D_117': 2, 'D_120': 1, 'D_126': 2,  'D_66': 1, 'D_68': 1
    }

    MAP_DICT = {
        'D_63': {'CO': 1, 'CR': 2, 'CL': 3, 'XZ': 4, 'XM': 5, 'XL': 6 },
        'D_64': {'O': 2, 'U': 3, 'R': 4, '-1' : 1, },    
    }

    DTYPES = sample.dtypes.to_dict()

    for col in DTYPES:
        if DTYPES[col] == np.float64:
            DTYPES[col] = np.float32
            
    # save feathers
    batch_process_fn('../input/amex-default-prediction/train_data.csv', 2000000, './data/train.feather')
    
    partitions = [

        (0, 2681973), 
        (2681973, 5362891), 
        (5362891, 8045043), 
        (8045043, None)

    ]

    for n, (start, stop) in enumerate(partitions):
        batch_process_fn(
            '../input/amex-default-prediction/test_data.csv', 2000000, 
            './data/test_part{0}.feather'.format(n), 
            start=start, stop=stop
        )
    
"""
with open('preprocess_script.py', 'w') as f:
    f.writelines(script)

In [ ]:
%%time
PREPROCESSING = False

if PREPROCESSING:
    !{sys.executable} preprocess_script.py
    DATA_DIR = './data'
else:
    DATA_DIR = '../input/amexsavedasfeather/'

### Feature extraction

Here we create aggregated dataset. To simplify the process I used the features from [this Kernel](https://www.kaggle.com/code/huseyincot/amex-agg-data-how-it-created) proposed by [huseyincotel](https://www.kaggle.com/huseyincot), just speed up execution with **CuDF**

In [ ]:
def feature_extraction(path):
    """
    Simple features
    """
    data = cudf.read_feather(path)
    
    CATS = ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']
    NUMS = data.columns.drop(['customer_ID', 'S_2'] + CATS).to_list()
    
    agg_nums = data.groupby('customer_ID')[NUMS].agg(['mean', 'std', 'min', 'max', 'last']).astype(np.float32)
    agg_nums.columns = ['_'.join(x) for x in agg_nums.columns]
    
    agg_cats = data.groupby("customer_ID")[CATS].agg(['count', 'last', 'nunique']).astype(np.float32)
    agg_cats.columns = ['_'.join(x) for x in agg_cats.columns]
    
    data = cudf.concat([agg_nums, agg_cats, ], axis=1).to_pandas()
    
    return data

In [ ]:
%%time
# DataFrames
train = feature_extraction(os.path.join(DATA_DIR, 'train.feather'))
labels = pd.read_feather(os.path.join(DATA_DIR, 'target.feather')).set_index('my_id')

# features arrays
X = train.values
y = labels['target'].loc[train.index].values

### Metric comparison

Proposed by host metric implementation based on **pandas** looks not very efficient. Lets take some random (not nullable) feature as prediction and evaluate the time

In [ ]:
# lets create simple dummy predicion
dummy_pred = X[:, 10]

In [ ]:
def amex_metric_official(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:

    def top_four_percent_captured(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        df = (pd.concat([y_true, y_pred], axis='columns')
              .sort_values('prediction', ascending=False))
        df['weight'] = df['target'].apply(lambda x: 20 if x==0 else 1)
        four_pct_cutoff = int(0.04 * df['weight'].sum())
        df['weight_cumsum'] = df['weight'].cumsum()
        df_cutoff = df.loc[df['weight_cumsum'] <= four_pct_cutoff]
        return (df_cutoff['target'] == 1).sum() / (df['target'] == 1).sum()
        
    def weighted_gini(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        df = (pd.concat([y_true, y_pred], axis='columns')
              .sort_values('prediction', ascending=False))
        df['weight'] = df['target'].apply(lambda x: 20 if x==0 else 1)
        df['random'] = (df['weight'] / df['weight'].sum()).cumsum()
        total_pos = (df['target'] * df['weight']).sum()
        df['cum_pos_found'] = (df['target'] * df['weight']).cumsum()
        df['lorentz'] = df['cum_pos_found'] / total_pos
        df['gini'] = (df['lorentz'] - df['random']) * df['weight']
        return df['gini'].sum()

    def normalized_weighted_gini(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        y_true_pred = y_true.rename(columns={'target': 'prediction'})
        return weighted_gini(y_true, y_pred) / weighted_gini(y_true, y_true_pred)

    g = normalized_weighted_gini(y_true, y_pred)
    d = top_four_percent_captured(y_true, y_pred)

    return 0.5 * (g + d)

In [ ]:
# proposed implementation takes DataFrame as input, so lets do so
ydf = DataFrame({'target': y})
xdf = DataFrame({'prediction': dummy_pred})

In [ ]:
%%timeit
val = amex_metric_official(ydf, xdf)

In [ ]:
amex_metric_official(ydf, xdf)

Takes about 1 second. Not too long to evaluate the model quality, but almost impossible to use as early stopping detector for tree ensembles

### Numpy implementation

Implementation based. on **NumPy** looks much more efficient. It is proposed by [Konstantin Yakovlev](https://www.kaggle.com/kyakovlev) in this post [this post](https://www.kaggle.com/competitions/amex-default-prediction/discussion/327534). Lets see how fast it is.

In [ ]:
def amex_metric_numpy(y_true: np.array, y_pred: np.array) -> float:

    labels     = np.transpose(np.array([y_true, y_pred]))
    labels     = labels[labels[:, 1].argsort()[::-1]]
    weights    = np.where(labels[:,0]==0, 20, 1)
    cut_vals   = labels[np.cumsum(weights) <= int(0.04 * np.sum(weights))]
    top_four   = np.sum(cut_vals[:,0]) / np.sum(labels[:,0])

    gini = [0,0]
    for i in [1,0]:
        labels         = np.transpose(np.array([y_true, y_pred]))
        labels         = labels[labels[:, i].argsort()[::-1]]
        weight         = np.where(labels[:,0]==0, 20, 1)
        weight_random  = np.cumsum(weight / np.sum(weight))
        total_pos      = np.sum(labels[:, 0] *  weight)
        cum_pos_found  = np.cumsum(labels[:, 0] * weight)
        lorentz        = cum_pos_found / total_pos
        gini[i]        = np.sum((lorentz - weight_random) * weight)

    return 0.5 * (gini[1]/gini[0] + top_four)

In [ ]:
%%timeit
val = amex_metric_numpy(y, dummy_pred)

In [ ]:
amex_metric_numpy(y, dummy_pred)

##### Good job! Almost x5 speed up. But can we make it faster? :)

In [ ]:
def check_input(arr):
    """
    Check the input
    """
    if type(arr) is pd.DataFrame:
        arr = arr[arr.columns[0]]
        
    if type(arr) is pd.Series:
        arr = arr.values
        
    if len(arr.shape) > 1:
        arr = arr[:, 0]
        
    return arr


def gini(cs_0, cs_1, sum_0, sum_1):
    """
    Gini part
    """
    auc_ = (cs_0 - sum_0 / 2) * sum_1
    tot = cs_0[-1] * cs_1[-1]

    return 2 * float(auc_.sum() / tot) - 1


def recall_at4(cs_0, cs_1, sum_1):
    """
    Recall part
    """
    cs_tot = cs_0 + cs_1
    th = cs_tot[-1] * 0.96
    
    return float(sum_1[cs_tot >= th].sum() / cs_1[-1])
    
    
def amex_score(y_true, y_pred):
    """
    Faster NumPy metric implementation
    """
    y_true = check_input(y_true)
    y_pred = check_input(y_pred)

    sum_1 = y_true[y_pred.argsort()]
    sum_0 = (1 - sum_1) 
    sum_0 *= 20
    
    cs_0, cs_1 = np.cumsum(sum_0, dtype=np.float64), np.cumsum(sum_1, dtype=np.float64)
    
    g = gini(cs_0, cs_1, sum_0, sum_1)
    d = recall_at4(cs_0, cs_1, sum_1)
    
    return (g + d) / 2

In [ ]:
%%timeit
val = amex_score(y, dummy_pred)

In [ ]:
amex_score(y, dummy_pred)

##### This NumPy implementation is about x3 faster. But remember, this is GPU Kernel, so let's use it : )

In [ ]:
def amex_score_gpu(y_true, y_pred):
    """
    CuPy based AMEX metric
    """
    y_true = cp.asarray(check_input(y_true))
    y_pred = cp.asarray(check_input(y_pred))
    
    unique = cp.unique(y_pred)
    rank = cp.searchsorted(unique, y_pred)
    
    sum_1 = cp.zeros_like(unique, dtype=cp.float64)
    sum_1.scatter_add(rank, y_true)
    
    sum_0 = cp.zeros_like(unique, dtype=cp.float64)
    sum_0.scatter_add(rank, 1 - y_true)
    sum_0 *= 20
    
    cs_0, cs_1 = sum_0.cumsum(), sum_1.cumsum()
    
    g = gini(cs_0, cs_1, sum_0, sum_1)
    d = recall_at4(cs_0, cs_1, sum_1)
    
    return (g + d) / 2

In [ ]:
%%timeit
val = amex_score_gpu(y, dummy_pred)

In [ ]:
amex_score_gpu(y, dummy_pred)

##### And x5 more using GPU. Some time is spent for CPU->GPU data transfer, so if target and predictions already will be on the device, it will be a liite bit faster

In [ ]:
ygpu = cp.asarray(y)
xgpu = cp.asarray(dummy_pred)

In [ ]:
%%timeit
val = amex_score_gpu(ygpu, xgpu)

So why do we need so fast metric? The answer is - to perform early stopping in Gradient Boosting. And here is what I want to show

### Py-Boost

**Py-Boost** is GBDT implementation written only on Python. Despite this fact, it is fast because it is based on Python GPU libraries (**CuPy** mostly). I made it few month ago mainly for the research purposes, that is focused on multioutput training (multilabel/multiclass tasks), where **py-boost** is very efficient. But we can also train a simple binary task too. 

To learn more, visit our [Github repo](https://github.com/sb-ai-lab/Py-Boost). Here you will find some more usage tutorials. If you like this tool, you also can star us :)

During **Py-Boost** traing data is represented as GPU arrays, so we can loosely pass  **amex_score_gpu** function for the fast evaluation with no overhead on data transfer, just need to write the wrapper class

In [ ]:
class CustomAMEXMetric(Metric):
    """Custom AMEX Metric for Py-Boost"""
    
    def compare(self, v0 ,v1):
        """
        It should return True if v0 (new value) metric value is better than v1 (old value), False othewise
        """
        return v0 > v1
    
    def __call__(self, y_true, y_pred, sample_weight=None):
        """
        We also update __call__ method with AMEX score
        """
        return amex_score_gpu(y_true, y_pred)

##### Other **py-boost** hyperparameters and interface are very common for all GBDT users. Here is the example of Cross-Validation loop with custom AMEX metric:

In [ ]:
params = {

    'loss': 'bce',
    'metric': CustomAMEXMetric(),
    'ntrees': 10000,
    'lr': 0.01,
    'min_gain_to_split': 0,
    'lambda_l2': 10,
    'max_bin': 256,
    'max_depth': 6,
    'min_data_in_leaf': 10,
    'colsample': 0.9,
    'subsample': 0.6,
    'quant_sample': 1000000,
    'es': 300,
    'verbose': 100,
    
}

def cv_loop(params, X, y, score=amex_score):
    """
    CrossValidation loop for Py-Boost training
    """
    scores = []
    models = []
    indices = np.arange(X.shape[0])
    oof_pred = np.zeros((X.shape[0], ), dtype=np.float32)
    cv = StratifiedKFold(5, shuffle=True, random_state=42)
    
    for n, (f0, f1) in enumerate(cv.split(y, y)):
        
        X_test, y_test = X[f1], y[f1]
        
        model = GradientBoosting(**params)
        model.fit(X[f0], y[f0], eval_sets=[{'X': X_test, 'y': y_test}])
        
        models.append(model)
        oof_pred[f1] = model.predict(X_test, batch_size=1000000)[:, 0]
        scores.append(
            score(y_test, oof_pred[f1])
        )
        
        print('Fold {0}: score {1}'.format(n, scores[-1]))
        
    return oof_pred, np.array(scores), models

In [ ]:
%%time
oof_pred, scores, models = cv_loop(params, X, y)

In [ ]:
amex_score(y, oof_pred)

### Feature importance

As usual, you can visualize feature importances

In [ ]:
Series(models[0].get_feature_importance(), index=train.columns).sort_values(
    ascending=False)[:20][::-1].plot(kind='barh')

In [ ]:
del train, X

### Inference

Inference code is also quite simple

In [ ]:
def inference(flist, models):
    """
    Inference separated by partitions
    """
    pred_parts = []

    for path in flist:
        test = feature_extraction(path)
        X_test, index = test.values, test.index

        del test

        pred = np.zeros((X_test.shape[0], ), dtype=np.float32)

        for model in models:
            pred += model.predict(X_test, batch_size=5000000)[:, 0]

        pred /= len(models)
        pred_parts.append(Series(pred, index=index))

        del X_test
        print('{0} finished'.format(path))    
        
    return pd.concat(pred_parts, axis=0)

In [ ]:
%%time
prediction = inference(
    [os.path.join(DATA_DIR, 'test_part{0}.feather'.format(x)) for x in range(4)],
    models
)

The last thing to do is generating submission

In [ ]:
ssub = pd.read_feather(os.path.join(DATA_DIR, 'ssub.feather')).set_index('my_id')
ssub['prediction'] = prediction

In [ ]:
ssub[['customer_ID', 'prediction']]

In [ ]:
ssub[['customer_ID', 'prediction']].to_csv('submission.csv', index=False)

#### Hope this kernel was helpful. Thanks for your attention! And Good luck :)